In [1]:
import netCDF4 as nc
import numpy
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)
import os
import cartopy.feature as cfeature
import glob
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib import cm
#import seaborn as sns
import cartopy.crs as ccrs
import cartopy.feature as cf
import mplotutils as mpu # helper functions for cartopy and matplotlib
#import regionmask
from func_plots import *
from func_stats import *

In [2]:
# Regional plot with coastline and country borders
def format_axes(axes):
    for i, ax in enumerate(axes):
        #ax.set_title(plot_names[i])
        ax.coastlines(resolution='50m', linewidth=0.45)
        # ax.add_feature(cf.BORDERS, linewidth=0.3)
        ax.set_extent([lonmin, lonmax, latmin, latmax], crs=data_proj)
        ax.gridlines(draw_labels=False, linewidth=0.3, color="gray", xlocs=range(-180, 180, 10), ylocs=range(-90, 90, 10))

def add_headers(fig, *, row_headers=None, col_headers=None, row_pad=1, col_pad=4, rotate_row_headers=True, **text_kwargs):
    # Based on https://stackoverflow.com/a/25814386
    axes = fig.get_axes()

    for ax in axes:
        sbs = ax.get_subplotspec()

        # Putting headers on cols
        if (col_headers is not None) and sbs.is_first_row():
            ax.annotate(
                col_headers[sbs.colspan.start],
                xy=(0.5, 1),
                xytext=(0, col_pad),
                xycoords="axes fraction",
                textcoords="offset points",
                ha="center",
                va="baseline",
                **text_kwargs,
            )

        # Putting headers on rows
        if (row_headers is not None) and sbs.is_first_col():
            ax.annotate(
                row_headers[sbs.rowspan.start],
                xy=(0, 0.5),
                xytext=(-ax.yaxis.labelpad - row_pad, 0),
                xycoords=ax.transAxes, # ax.yaxis.label replaced by ax.transAxes
                textcoords="offset points",
                ha="center",
                va="center", # vertical even after rotation
                rotation=rotate_row_headers * 90,
                **text_kwargs,
            )

lonmin, lonmax, latmin, latmax = [-11, 37, 35, 70.5]                   # window for plotting
set_lonmin, set_lonmax, set_latmin, set_latmax = [-35, 65, 30, 72.6]   # subset the data to get sensible vmin and vmax for the colorbar
data_proj = ccrs.PlateCarree()
map_proj = ccrs.LambertConformal(central_longitude=15) # for regional maps

In [5]:
filepath = '/landclim2/yiyaoy/COSMO-CLM2-simulations/timmean/clm5.0_eur0.5_control_h0_2020-2024.nc_timmean'
with nc.Dataset(filepath) as ds:
    lat = ds.variables['lat'][:]       # already a NumPy array
    lon = ds.variables['lon'][:]       # likewise

# shift longitudes from [0,360] to [-180,180]
lon = np.where(lon > 180, lon - 360, lon)

In [81]:
def _load_case(var, lat, lon, scenario, season_suffix, subdir):
    """
    Helper to load one file, mask >1e9, and wrap in an xarray.DataArray.
    """
    fn = (
        f"/landclim2/yiyaoy/COSMO-CLM2-simulations/"
        f"{subdir}/post-processing/{'timmean' if season_suffix == '' else 'seasonal/timmean'}"
        f"/mergetime/"
        f"clm5.0_eur0.5_{scenario}_h0_2025-2059"
        f"{season_suffix}.nc_timmean_timmean"
    )
    ds = nc.Dataset(fn)
    arr = np.squeeze(np.array(ds.variables[var][:]))
    arr[arr > 1e9] = np.nan
    return xr.DataArray(arr, coords={'y': lat, 'x': lon}, dims=['y', 'x'])

def get_mean_data_aff_afb(var, lat, lon):
    """
    Returns an xarray.Dataset with one DataArray per scenario-season.
    Keys will be e.g. 'control', 'control_JJA', 'all_forest_MAM', etc.
    """
    cases = [
        # (scenario-name, season-suffix, top‑level folder)
        ('control',      '',     'control/clm5'),
        ('control',      '_JJA', 'control/clm5'),
        ('control',      '_MAM', 'control/clm5'),
        ('all_forest',   '',     'all_forest/clm5'),
        ('all_forest',   '_JJA', 'all_forest/clm5'),
        ('all_forest',   '_MAM', 'all_forest/clm5'),
        ('all_grass',   '',      'all_grass/clm5'),
        ('all_grass',   '_JJA',  'all_grass/clm5'),
        ('all_grass',   '_MAM',  'all_grass/clm5'),
        ('all_forest_broadleaf',    '',     'all_forest_broadleaf/clm5'),
        ('all_forest_broadleaf',    '_JJA', 'all_forest_broadleaf/clm5'),
        ('all_forest_broadleaf',    '_MAM', 'all_forest_broadleaf/clm5'),
    ]

    data_vars = {}
    for scen, season, subdir in cases:
        key = f"{scen}{season}"
        data_vars[key] = _load_case(var, lat, lon, scen, season, subdir)
    
    return xr.Dataset(data_vars)

In [82]:
tair = get_mean_data_aff_afb('TSA', lat, lon)
tsfc = get_mean_data_aff_afb('TSKIN', lat, lon)
tatm = get_mean_data_aff_afb('TBOT', lat, lon)

In [ ]:
# 1. Levels, tick‑labels and cmaps
levels_dict = {
    'seq': np.array([-5, 0, 5, 10, 15, 20, 25, 30]) + 273.15,
    'div': np.array([-2, -1, -0.5, -0.3, -0.1, 0.1, 0.3, 0.5, 1, 2])
}
ticks_dict = {
    "seq": ["\N{MINUS SIGN}5", "0", "5", "10", "15", "20", "25", "30"],
    "div": ['\N{MINUS SIGN}2.0', '\N{MINUS SIGN}1.0', '\N{MINUS SIGN}0.5', '\N{MINUS SIGN}0.3', '\N{MINUS SIGN}0.1', '+0.1', '+0.3', '+0.5', '+1.0', '+2.0']
}
cmaps = {'seq': 'hot_r', 'div': 'RdBu_r'}

unit = '$\\mathregular{^\\circ C}$'
vmin, vmax, extend = 0, 42, 'both'

# 2. Define each panel: (row, col), (var1, var0, levels‑type), title, letter
panels = [
    # row 0
    (0, 0, ('all_forest','control'),      'div', 'Aff–Ctl: $\\Delta$ $\mathregular{T_{sfc,yearM}}$', 'a'),
    (0, 1, ('all_forest_broadleaf','control'),'div','AfB–Ctl: $\\Delta$ $\mathregular{T_{sfc,yearM}}$','b'),
    (0, 2, ('all_forest_broadleaf','all_forest'),'div','AfB–Aff: $\\Delta$ $\mathregular{T_{sfc,yearM}}$','c'),
    (0, 3, ('all_forest_broadleaf','all_grass'),'div','AfB–Def: $\\Delta$ $\mathregular{T_{sfc,yearM}}$','d'),
    
    # row 1
    (1, 0, ('all_forest','control'),      'div', 'Aff–Ctl: $\\Delta$ $\mathregular{T_{air,yearM}}$','e'),
    (1, 1, ('all_forest_broadleaf','control'),'div','AfB–Ctl: $\\Delta$ $\mathregular{T_{air,yearM}}$','f'),
    (1, 2, ('all_forest_broadleaf','all_forest'),'div','AfB–Aff: $\\Delta$ $\mathregular{T_{air,yearM}}$','g'),
    (1, 3, ('all_forest_broadleaf','all_grass'),'div','AfB–Def: $\\Delta$ $\mathregular{T_{air,yearM}}$','h'),
    
    # row 2
    (2, 0, ('all_forest','control'),      'div', 'Aff–Ctl: $\\Delta$ $\mathregular{T_{atm,yearM}}$','i'),
    (2, 1, ('all_forest_broadleaf','control'),'div','AfB–Ctl: $\\Delta$ $\mathregular{T_{atm,yearM}}$','j'),
    (2, 2, ('all_forest_broadleaf','all_forest'),'div','AfB–Aff: $\\Delta$ $\mathregular{T_{atm,yearM}}$','k'),
    (2, 3, ('all_forest_broadleaf','all_grass'),'div','AfB–Def: $\\Delta$ $\mathregular{T_{atm,yearM}}$','l'),

]

# 3. Make the figure & axes grid
fig, axes = plt.subplots(
    3, 4,
    figsize=(12, 8),
    subplot_kw=dict(projection=map_proj, facecolor="lightgrey"),
    dpi=300
)
format_axes(axes.flatten())
fig.subplots_adjust(hspace=0.2, wspace=0.1, left=0.03, right=0.935, bottom=0.03, top=0.9)

# 4. Loop and plot
for row, col, (high, low), lvl_type, title, letter in panels:
    ax = axes[row, col]
    levels = levels_dict[lvl_type]
    cmap   = cmaps[lvl_type]
    ticks  = ticks_dict[lvl_type]

    diff = tsfc[high] - tsfc[low]  \
           if 'sfc' in title else \
           (tair if 'air' in title else tatm)[high] - (tair if 'air' in title else tatm)[low]

    opt = dict(
        transform=data_proj,
        vmin=vmin, vmax=vmax,
        levels=levels,
        extend=extend,
        add_colorbar=False,
        cmap=cmap
    )

    h = diff.plot(ax=ax, **opt)
    ax.set_title(f"{title} ({unit})")
    ax.add_feature(cfeature.OCEAN, color='whitesmoke')

    # only draw colorbar on rightmost column of each row
    if col <= 3:
        cbar = mpu.colorbar(
            h, ax,
            orientation='vertical',
            size=0.04,
            extend=extend,
            pad=0.05,
            ticks=levels
        )
        cbar.set_ticklabels(ticks)

    ax.text(
        0.02, 0.93, letter,
        color='k', fontsize=10,
        transform=ax.transAxes, weight='bold'
    )

In [ ]:
# 1. Levels, tick‑labels and cmaps
levels_dict = {
    'seq': np.array([-5, 0, 5, 10, 15, 20, 25, 30]) + 273.15,
    'div': np.array([-2, -1, -0.5, -0.3, -0.1, 0.1, 0.3, 0.5, 1, 2])
}
ticks_dict = {
    "seq": ["\N{MINUS SIGN}5", "0", "5", "10", "15", "20", "25", "30"],
    "div": ['\N{MINUS SIGN}2.0', '\N{MINUS SIGN}1.0', '\N{MINUS SIGN}0.5', '\N{MINUS SIGN}0.3', '\N{MINUS SIGN}0.1', '+0.1', '+0.3', '+0.5', '+1.0', '+2.0']
}
cmaps = {'seq': 'hot_r', 'div': 'RdBu_r'}

unit = '$\\mathregular{^\\circ C}$'
vmin, vmax, extend = 0, 42, 'both'

# 2. Define each panel: (row, col), (var1, var0, levels‑type), title, letter
panels = [
    # row 0
    (0, 0, ('all_forest_JJA','control_JJA'),      'div', 'Aff–Ctl: $\\Delta$ $\mathregular{T_{sfc,jjaM}}$', 'a'),
    (0, 1, ('all_forest_broadleaf_JJA','control_JJA'),'div','AfB–Ctl: $\\Delta$ $\mathregular{T_{sfc,jjaM}}$','b'),
    (0, 2, ('all_forest_broadleaf_JJA','all_forest_JJA'),'div','AfB–Aff: $\\Delta$ $\mathregular{T_{sfc,jjaM}}$','c'),
    (0, 3, ('all_forest_broadleaf_JJA','all_grass_JJA'),'div','AfB–Def: $\\Delta$ $\mathregular{T_{sfc,jjaM}}$','d'),
    
    # row 1
    (1, 0, ('all_forest_JJA','control_JJA'),      'div', 'Aff–Ctl: $\\Delta$ $\mathregular{T_{air,jjaM}}$','e'),
    (1, 1, ('all_forest_broadleaf_JJA','control_JJA'),'div','AfB–Ctl: $\\Delta$ $\mathregular{T_{air,jjaM}}$','f'),
    (1, 2, ('all_forest_broadleaf_JJA','all_forest_JJA'),'div','AfB–Aff: $\\Delta$ $\mathregular{T_{air,jjaM}}$','g'),
    (1, 3, ('all_forest_broadleaf_JJA','all_grass_JJA'),'div','AfB–Def: $\\Delta$ $\mathregular{T_{air,jjaM}}$','h'),
    
    # row 2
    (2, 0, ('all_forest_JJA','control_JJA'),      'div', 'Aff–Ctl: $\\Delta$ $\mathregular{T_{atm,jjaM}}$','i'),
    (2, 1, ('all_forest_broadleaf_JJA','control_JJA'),'div','AfB–Ctl: $\\Delta$ $\mathregular{T_{atm,jjaM}}$','j'),
    (2, 2, ('all_forest_broadleaf_JJA','all_forest_JJA'),'div','AfB–Aff: $\\Delta$ $\mathregular{T_{atm,jjaM}}$','k'),
    (2, 3, ('all_forest_broadleaf_JJA','all_grass_JJA'),'div','AfB–Def: $\\Delta$ $\mathregular{T_{atm,jjaM}}$','l'),
]

# 3. Make the figure & axes grid
fig, axes = plt.subplots(
    3, 4,
    figsize=(12, 8),
    subplot_kw=dict(projection=map_proj, facecolor="lightgrey"),
    dpi=300
)
format_axes(axes.flatten())
fig.subplots_adjust(hspace=0.2, wspace=0.1, left=0.03, right=0.935, bottom=0.03, top=0.9)

# 4. Loop and plot
for row, col, (high, low), lvl_type, title, letter in panels:
    ax = axes[row, col]
    levels = levels_dict[lvl_type]
    cmap   = cmaps[lvl_type]
    ticks  = ticks_dict[lvl_type]

    diff = tsfc[high] - tsfc[low]  \
           if 'sfc' in title else \
           (tair if 'air' in title else tatm)[high] - (tair if 'air' in title else tatm)[low]

    opt = dict(
        transform=data_proj,
        vmin=vmin, vmax=vmax,
        levels=levels,
        extend=extend,
        add_colorbar=False,
        cmap=cmap
    )

    h = diff.plot(ax=ax, **opt)
    ax.set_title(f"{title} ({unit})")
    ax.add_feature(cfeature.OCEAN, color='whitesmoke')

    # only draw colorbar on rightmost column of each row
    if col <= 3:
        cbar = mpu.colorbar(
            h, ax,
            orientation='vertical',
            size=0.04,
            extend=extend,
            pad=0.05,
            ticks=levels
        )
        cbar.set_ticklabels(ticks)

    ax.text(
        0.02, 0.93, letter,
        color='k', fontsize=10,
        transform=ax.transAxes, weight='bold'
    )

In [ ]:
# 1. Levels, tick‑labels and cmaps
levels_dict = {
    'seq': np.array([-5, 0, 5, 10, 15, 20, 25, 30]) + 273.15,
    'div': np.array([-2, -1, -0.5, -0.3, -0.1, 0.1, 0.3, 0.5, 1, 2])
}
ticks_dict = {
    "seq": ["\N{MINUS SIGN}5", "0", "5", "10", "15", "20", "25", "30"],
    "div": ['\N{MINUS SIGN}2.0', '\N{MINUS SIGN}1.0', '\N{MINUS SIGN}0.5', '\N{MINUS SIGN}0.3', '\N{MINUS SIGN}0.1', '+0.1', '+0.3', '+0.5', '+1.0', '+2.0']
}
cmaps = {'seq': 'hot_r', 'div': 'RdBu_r'}

unit = '$\\mathregular{^\\circ C}$'
vmin, vmax, extend = 0, 42, 'both'

# 2. Define each panel: (row, col), (var1, var0, levels‑type), title, letter
panels = [
    # row 0
    (0, 0, ('all_forest_MAM','control_MAM'),      'div', 'Aff–Ctl: $\\Delta$ $\mathregular{T_{sfc,mamM}}$', 'a'),
    (0, 1, ('all_forest_broadleaf_MAM','control_MAM'),'div','AfB–Ctl: $\\Delta$ $\mathregular{T_{sfc,mamM}}$','b'),
    (0, 2, ('all_forest_broadleaf_MAM','all_forest_MAM'),'div','AfB–Aff: $\\Delta$ $\mathregular{T_{sfc,mamM}}$','c'),
    (0, 3, ('all_forest_broadleaf_MAM','all_grass_MAM'),'div','AfB–Def: $\\Delta$ $\mathregular{T_{sfc,mamM}}$','d'),
    
    # row 1
    (1, 0, ('all_forest_MAM','control_MAM'),      'div', 'Aff–Ctl: $\\Delta$ $\mathregular{T_{air,mamM}}$','e'),
    (1, 1, ('all_forest_broadleaf_MAM','control_MAM'),'div','AfB–Ctl: $\\Delta$ $\mathregular{T_{air,mamM}}$','f'),
    (1, 2, ('all_forest_broadleaf_MAM','all_forest_MAM'),'div','AfB–Aff: $\\Delta$ $\mathregular{T_{air,mamM}}$','g'),
    (1, 3, ('all_forest_broadleaf_MAM','all_grass_MAM'),'div','AfB–Def: $\\Delta$ $\mathregular{T_{air,mamM}}$','h'),
    
    # row 2
    (2, 0, ('all_forest_MAM','control_MAM'),      'div', 'Aff–Ctl: $\\Delta$ $\mathregular{T_{atm,mamM}}$','i'),
    (2, 1, ('all_forest_broadleaf_MAM','control_MAM'),'div','AfB–Ctl: $\\Delta$ $\mathregular{T_{atm,mamM}}$','j'),
    (2, 2, ('all_forest_broadleaf_MAM','all_forest_MAM'),'div','AfB–Aff: $\\Delta$ $\mathregular{T_{atm,mamM}}$','k'),
    (2, 3, ('all_forest_broadleaf_MAM','all_grass_MAM'),'div','AfB–Def: $\\Delta$ $\mathregular{T_{atm,mamM}}$','l'),

]

# 3. Make the figure & axes grid
fig, axes = plt.subplots(
    3, 4,
    figsize=(12, 8),
    subplot_kw=dict(projection=map_proj, facecolor="lightgrey"),
    dpi=300
)
format_axes(axes.flatten())
fig.subplots_adjust(hspace=0.2, wspace=0.1, left=0.03, right=0.935, bottom=0.03, top=0.9)

# 4. Loop and plot
for row, col, (high, low), lvl_type, title, letter in panels:
    ax = axes[row, col]
    levels = levels_dict[lvl_type]
    cmap   = cmaps[lvl_type]
    ticks  = ticks_dict[lvl_type]

    diff = tsfc[high] - tsfc[low]  \
           if 'sfc' in title else \
           (tair if 'air' in title else tatm)[high] - (tair if 'air' in title else tatm)[low]

    opt = dict(
        transform=data_proj,
        vmin=vmin, vmax=vmax,
        levels=levels,
        extend=extend,
        add_colorbar=False,
        cmap=cmap
    )

    h = diff.plot(ax=ax, **opt)
    ax.set_title(f"{title} ({unit})")
    ax.add_feature(cfeature.OCEAN, color='whitesmoke')

    # only draw colorbar on rightmost column of each row
    if col <= 3:
        cbar = mpu.colorbar(
            h, ax,
            orientation='vertical',
            size=0.04,
            extend=extend,
            pad=0.05,
            ticks=levels
        )
        cbar.set_ticklabels(ticks)

    ax.text(
        0.02, 0.93, letter,
        color='k', fontsize=10,
        transform=ax.transAxes, weight='bold'
    )

In [93]:
def _load_case_max(var, lat, lon, scenario, season_suffix, subdir):
    """
    Helper to load one file, mask >1e9, and wrap in an xarray.DataArray.
    """
    fn = (
        f"/landclim2/yiyaoy/COSMO-CLM2-simulations/"
        f"{subdir}/post-processing/{'timmean' if season_suffix == '' else 'seasonal/timmean'}"
        f"/mergetime/"
        f"clm5.0_eur0.5_{scenario}_h2_2025-2059"
        f"{season_suffix}.nc_timmean_timmean"
    )
    ds = nc.Dataset(fn)
    arr = np.squeeze(np.array(ds.variables[var][:]))
    arr[arr > 1e9] = np.nan
    return xr.DataArray(arr, coords={'y': lat, 'x': lon}, dims=['y', 'x'])

def get_max_data_aff_afb(var, lat, lon):
    """
    Returns an xarray.Dataset with one DataArray per scenario-season.
    Keys will be e.g. 'control', 'control_JJA', 'all_forest_MAM', etc.
    """
    cases = [
        # (scenario-name, season-suffix, top‑level folder)
        ('control',      '',     'control/clm5'),
        ('control',      '_JJA', 'control/clm5'),
        ('control',      '_MAM', 'control/clm5'),
        ('all_forest',   '',     'all_forest/clm5'),
        ('all_forest',   '_JJA', 'all_forest/clm5'),
        ('all_forest',   '_MAM', 'all_forest/clm5'),
        ('all_grass',   '',     'all_grass/clm5'),
        ('all_grass',   '_JJA', 'all_grass/clm5'),
        ('all_grass',   '_MAM', 'all_grass/clm5'),
        ('all_forest_broadleaf',    '',     'all_forest_broadleaf/clm5'),
        ('all_forest_broadleaf',    '_JJA', 'all_forest_broadleaf/clm5'),
        ('all_forest_broadleaf',    '_MAM', 'all_forest_broadleaf/clm5'),
    ]

    data_vars = {}
    for scen, season, subdir in cases:
        key = f"{scen}{season}"
        data_vars[key] = _load_case_max(var, lat, lon, scen, season, subdir)
    
    return xr.Dataset(data_vars)

In [94]:
tair_X = get_max_data_aff_afb('TSA', lat, lon)
tsfc_X = get_max_data_aff_afb('TSKIN', lat, lon)
tatm_X = get_max_data_aff_afb('TBOT', lat, lon)

In [ ]:
# 1. Levels, tick‑labels and cmaps
levels_dict = {
    'seq': np.array([-5, 0, 5, 10, 15, 20, 25, 30]) + 273.15,
    'div': np.array([-2, -1, -0.5, -0.3, -0.1, 0.1, 0.3, 0.5, 1, 2])
}
ticks_dict = {
    "seq": ["\N{MINUS SIGN}5", "0", "5", "10", "15", "20", "25", "30"],
    "div": ['\N{MINUS SIGN}2.0', '\N{MINUS SIGN}1.0', '\N{MINUS SIGN}0.5', '\N{MINUS SIGN}0.3', '\N{MINUS SIGN}0.1', '+0.1', '+0.3', '+0.5', '+1.0', '+2.0']
}
cmaps = {'seq': 'hot_r', 'div': 'RdBu_r'}

unit = '$\\mathregular{^\\circ C}$'
vmin, vmax, extend = 0, 42, 'both'

# 2. Define each panel: (row, col), (var1, var0, levels‑type), title, letter
panels = [
    # row 0
    (0, 0, ('all_forest','control'),      'div', 'Aff–Ctl: $\\Delta$ $\mathregular{T_{sfc,yearX}}$', 'a'),
    (0, 1, ('all_forest_broadleaf','control'),'div','AfB–Ctl: $\\Delta$ $\mathregular{T_{sfc,yearX}}$','b'),
    (0, 2, ('all_forest_broadleaf','all_forest'),'div','AfB–Aff: $\\Delta$ $\mathregular{T_{sfc,yearX}}$','c'),
    (0, 3, ('all_forest_broadleaf','all_grass'),'div','AfB–Def: $\\Delta$ $\mathregular{T_{sfc,yearX}}$','d'),
    
    # row 1
    (1, 0, ('all_forest','control'),      'div', 'Aff–Ctl: $\\Delta$ $\mathregular{T_{air,yearX}}$','e'),
    (1, 1, ('all_forest_broadleaf','control'),'div','AfB–Ctl: $\\Delta$ $\mathregular{T_{air,yearX}}$','f'),
    (1, 2, ('all_forest_broadleaf','all_forest'),'div','AfB–Aff: $\\Delta$ $\mathregular{T_{air,yearX}}$','g'),
    (1, 3, ('all_forest_broadleaf','all_grass'),'div','AfB–Def: $\\Delta$ $\mathregular{T_{air,yearX}}$','h'),
    
    # row 2
    (2, 0, ('all_forest','control'),      'div', 'Aff–Ctl: $\\Delta$ $\mathregular{T_{atm,yearX}}$','i'),
    (2, 1, ('all_forest_broadleaf','control'),'div','AfB–Ctl: $\\Delta$ $\mathregular{T_{atm,yearX}}$','j'),
    (2, 2, ('all_forest_broadleaf','all_forest'),'div','AfB–Aff: $\\Delta$ $\mathregular{T_{atm,yearX}}$','k'),
    (2, 3, ('all_forest_broadleaf','all_grass'),'div','AfB–Def: $\\Delta$ $\mathregular{T_{atm,yearX}}$','l'),

]

# 3. Make the figure & axes grid
fig, axes = plt.subplots(
    3, 4,
    figsize=(12, 8),
    subplot_kw=dict(projection=map_proj, facecolor="lightgrey"),
    dpi=300
)
format_axes(axes.flatten())
fig.subplots_adjust(hspace=0.2, wspace=0.1, left=0.03, right=0.935, bottom=0.03, top=0.9)

# 4. Loop and plot
for row, col, (high, low), lvl_type, title, letter in panels:
    ax = axes[row, col]
    levels = levels_dict[lvl_type]
    cmap   = cmaps[lvl_type]
    ticks  = ticks_dict[lvl_type]

    diff = tsfc_X[high] - tsfc_X[low]  \
           if 'sfc' in title else \
           (tair_X if 'air' in title else tatm_X)[high] - (tair_X if 'air' in title else tatm_X)[low]

    opt = dict(
        transform=data_proj,
        vmin=vmin, vmax=vmax,
        levels=levels,
        extend=extend,
        add_colorbar=False,
        cmap=cmap
    )

    h = diff.plot(ax=ax, **opt)
    ax.set_title(f"{title} ({unit})")
    ax.add_feature(cfeature.OCEAN, color='whitesmoke')

    # only draw colorbar on rightmost column of each row
    if col <= 3:
        cbar = mpu.colorbar(
            h, ax,
            orientation='vertical',
            size=0.04,
            extend=extend,
            pad=0.05,
            ticks=levels
        )
        cbar.set_ticklabels(ticks)

    ax.text(
        0.02, 0.93, letter,
        color='k', fontsize=10,
        transform=ax.transAxes, weight='bold'
    )

In [ ]:
# 1. Levels, tick‑labels and cmaps
levels_dict = {
    'seq': np.array([-5, 0, 5, 10, 15, 20, 25, 30]) + 273.15,
    'div': np.array([-2, -1, -0.5, -0.3, -0.1, 0.1, 0.3, 0.5, 1, 2])
}
ticks_dict = {
    "seq": ["\N{MINUS SIGN}5", "0", "5", "10", "15", "20", "25", "30"],
    "div": ['\N{MINUS SIGN}2.0', '\N{MINUS SIGN}1.0', '\N{MINUS SIGN}0.5', '\N{MINUS SIGN}0.3', '\N{MINUS SIGN}0.1', '+0.1', '+0.3', '+0.5', '+1.0', '+2.0']
}
cmaps = {'seq': 'hot_r', 'div': 'RdBu_r'}

unit = '$\\mathregular{^\\circ C}$'
vmin, vmax, extend = 0, 42, 'both'

# 2. Define each panel: (row, col), (var1, var0, levels‑type), title, letter
panels = [
    # row 0
    (0, 0, ('all_forest_JJA','control_JJA'),      'div', 'Aff–Ctl: $\\Delta$ $\mathregular{T_{sfc,jjaX}}$', 'a'),
    (0, 1, ('all_forest_broadleaf_JJA','control_JJA'),'div','AfB–Ctl: $\\Delta$ $\mathregular{T_{sfc,jjaX}}$','b'),
    (0, 2, ('all_forest_broadleaf_JJA','all_forest_JJA'),'div','AfB–Aff: $\\Delta$ $\mathregular{T_{sfc,jjaX}}$','c'),
    (0, 3, ('all_forest_broadleaf_JJA','all_grass_JJA'),'div','AfB–Def: $\\Delta$ $\mathregular{T_{sfc,jjaX}}$','d'),
    
    # row 1
    (1, 0, ('all_forest_JJA','control_JJA'),      'div', 'Aff–Ctl: $\\Delta$ $\mathregular{T_{air,jjaX}}$','e'),
    (1, 1, ('all_forest_broadleaf_JJA','control_JJA'),'div','AfB–Ctl: $\\Delta$ $\mathregular{T_{air,jjaX}}$','f'),
    (1, 2, ('all_forest_broadleaf_JJA','all_forest_JJA'),'div','AfB–Aff: $\\Delta$ $\mathregular{T_{air,jjaX}}$','g'),
    (1, 3, ('all_forest_broadleaf_JJA','all_grass_JJA'),'div','AfB–Def: $\\Delta$ $\mathregular{T_{air,jjaX}}$','h'),
    
    # row 2
    (2, 0, ('all_forest_JJA','control_JJA'),      'div', 'Aff–Ctl: $\\Delta$ $\mathregular{T_{atm,jjaX}}$','i'),
    (2, 1, ('all_forest_broadleaf_JJA','control_JJA'),'div','AfB–Ctl: $\\Delta$ $\mathregular{T_{atm,jjaX}}$','j'),
    (2, 2, ('all_forest_broadleaf_JJA','all_forest_JJA'),'div','AfB–Aff: $\\Delta$ $\mathregular{T_{atm,jjaX}}$','k'),
    (2, 3, ('all_forest_broadleaf_JJA','all_grass_JJA'),'div','AfB–Def: $\\Delta$ $\mathregular{T_{atm,jjaX}}$','l'),

]

# 3. Make the figure & axes grid
fig, axes = plt.subplots(
    3, 4,
    figsize=(12, 8),
    subplot_kw=dict(projection=map_proj, facecolor="lightgrey"),
    dpi=300
)
format_axes(axes.flatten())
fig.subplots_adjust(hspace=0.2, wspace=0.1, left=0.03, right=0.935, bottom=0.03, top=0.9)

# 4. Loop and plot
for row, col, (high, low), lvl_type, title, letter in panels:
    ax = axes[row, col]
    levels = levels_dict[lvl_type]
    cmap   = cmaps[lvl_type]
    ticks  = ticks_dict[lvl_type]

    diff = tsfc_X[high] - tsfc_X[low]  \
           if 'sfc' in title else \
           (tair_X if 'air' in title else tatm_X)[high] - (tair_X if 'air' in title else tatm_X)[low]

    opt = dict(
        transform=data_proj,
        vmin=vmin, vmax=vmax,
        levels=levels,
        extend=extend,
        add_colorbar=False,
        cmap=cmap
    )

    h = diff.plot(ax=ax, **opt)
    ax.set_title(f"{title} ({unit})")
    ax.add_feature(cfeature.OCEAN, color='whitesmoke')

    # only draw colorbar on rightmost column of each row
    if col <= 3:
        cbar = mpu.colorbar(
            h, ax,
            orientation='vertical',
            size=0.04,
            extend=extend,
            pad=0.05,
            ticks=levels
        )
        cbar.set_ticklabels(ticks)

    ax.text(
        0.02, 0.93, letter,
        color='k', fontsize=10,
        transform=ax.transAxes, weight='bold'
    )

In [ ]:
# 1. Levels, tick‑labels and cmaps
levels_dict = {
    'seq': np.array([-5, 0, 5, 10, 15, 20, 25, 30]) + 273.15,
    'div': np.array([-2, -1, -0.5, -0.3, -0.1, 0.1, 0.3, 0.5, 1, 2])
}
ticks_dict = {
    "seq": ["\N{MINUS SIGN}5", "0", "5", "10", "15", "20", "25", "30"],
    "div": ['\N{MINUS SIGN}2.0', '\N{MINUS SIGN}1.0', '\N{MINUS SIGN}0.5', '\N{MINUS SIGN}0.3', '\N{MINUS SIGN}0.1', '+0.1', '+0.3', '+0.5', '+1.0', '+2.0']
}
cmaps = {'seq': 'hot_r', 'div': 'RdBu_r'}

unit = '$\\mathregular{^\\circ C}$'
vmin, vmax, extend = 0, 42, 'both'

# 2. Define each panel: (row, col), (var1, var0, levels‑type), title, letter
panels = [
    # row 0
    (0, 0, ('all_forest_MAM','control_MAM'),      'div', 'Aff–Ctl: $\\Delta$ $\mathregular{T_{sfc,mamX}}$', 'a'),
    (0, 1, ('all_forest_broadleaf_MAM','control_MAM'),'div','AfB–Ctl: $\\Delta$ $\mathregular{T_{sfc,mamX}}$','b'),
    (0, 2, ('all_forest_broadleaf_MAM','all_forest_MAM'),'div','AfB–Aff: $\\Delta$ $\mathregular{T_{sfc,mamX}}$','c'),
    (0, 3, ('all_forest_broadleaf_MAM','all_grass_MAM'),'div','AfB–Def: $\\Delta$ $\mathregular{T_{sfc,mamX}}$','d'),
    
    # row 1
    (1, 0, ('all_forest_MAM','control_MAM'),      'div', 'Aff–Ctl: $\\Delta$ $\mathregular{T_{air,mamX}}$','e'),
    (1, 1, ('all_forest_broadleaf_MAM','control_MAM'),'div','AfB–Ctl: $\\Delta$ $\mathregular{T_{air,mamX}}$','f'),
    (1, 2, ('all_forest_broadleaf_MAM','all_forest_MAM'),'div','AfB–Aff: $\\Delta$ $\mathregular{T_{air,mamX}}$','g'),
    (1, 3, ('all_forest_broadleaf_MAM','all_grass_MAM'),'div','AfB–Def: $\\Delta$ $\mathregular{T_{air,mamX}}$','h'),
    
    # row 2
    (2, 0, ('all_forest_MAM','control_MAM'),      'div', 'Aff–Ctl: $\\Delta$ $\mathregular{T_{atm,mamX}}$','i'),
    (2, 1, ('all_forest_broadleaf_MAM','control_MAM'),'div','AfB–Ctl: $\\Delta$ $\mathregular{T_{atm,mamX}}$','j'),
    (2, 2, ('all_forest_broadleaf_MAM','all_forest_MAM'),'div','AfB–Aff: $\\Delta$ $\mathregular{T_{atm,mamX}}$','k'),
    (2, 3, ('all_forest_broadleaf_MAM','all_grass_MAM'),'div','AfB–Def: $\\Delta$ $\mathregular{T_{atm,mamX}}$','l'),

]

# 3. Make the figure & axes grid
fig, axes = plt.subplots(
    3, 4,
    figsize=(12, 8),
    subplot_kw=dict(projection=map_proj, facecolor="lightgrey"),
    dpi=300
)
format_axes(axes.flatten())
fig.subplots_adjust(hspace=0.2, wspace=0.1, left=0.03, right=0.935, bottom=0.03, top=0.9)

# 4. Loop and plot
for row, col, (high, low), lvl_type, title, letter in panels:
    ax = axes[row, col]
    levels = levels_dict[lvl_type]
    cmap   = cmaps[lvl_type]
    ticks  = ticks_dict[lvl_type]

    diff = tsfc_X[high] - tsfc_X[low]  \
           if 'sfc' in title else \
           (tair_X if 'air' in title else tatm_X)[high] - (tair_X if 'air' in title else tatm_X)[low]

    opt = dict(
        transform=data_proj,
        vmin=vmin, vmax=vmax,
        levels=levels,
        extend=extend,
        add_colorbar=False,
        cmap=cmap
    )

    h = diff.plot(ax=ax, **opt)
    ax.set_title(f"{title} ({unit})")
    ax.add_feature(cfeature.OCEAN, color='whitesmoke')

    # only draw colorbar on rightmost column of each row
    if col <= 3:
        cbar = mpu.colorbar(
            h, ax,
            orientation='vertical',
            size=0.04,
            extend=extend,
            pad=0.05,
            ticks=levels
        )
        cbar.set_ticklabels(ticks)

    ax.text(
        0.02, 0.93, letter,
        color='k', fontsize=10,
        transform=ax.transAxes, weight='bold'
    )

In [98]:
def _load_case_min(var, lat, lon, scenario, season_suffix, subdir):
    """
    Helper to load one file, mask >1e9, and wrap in an xarray.DataArray.
    """
    fn = (
        f"/landclim2/yiyaoy/COSMO-CLM2-simulations/"
        f"{subdir}/post-processing/{'timmean' if season_suffix == '' else 'seasonal/timmean'}"
        f"/mergetime/"
        f"clm5.0_eur0.5_{scenario}_h4_2025-2059"
        f"{season_suffix}.nc_timmean_timmean"
    )
    ds = nc.Dataset(fn)
    arr = np.squeeze(np.array(ds.variables[var][:]))
    arr[arr > 1e9] = np.nan
    return xr.DataArray(arr, coords={'y': lat, 'x': lon}, dims=['y', 'x'])

def get_min_data_aff_afb(var, lat, lon):
    """
    Returns an xarray.Dataset with one DataArray per scenario-season.
    Keys will be e.g. 'control', 'control_JJA', 'all_forest_MAM', etc.
    """
    cases = [
        # (scenario-name, season-suffix, top‑level folder)
        ('control',      '',     'control/clm5'),
        ('control',      '_JJA', 'control/clm5'),
        ('control',      '_MAM', 'control/clm5'),
        ('all_forest',   '',     'all_forest/clm5'),
        ('all_forest',   '_JJA', 'all_forest/clm5'),
        ('all_forest',   '_MAM', 'all_forest/clm5'),
        ('all_grass',   '',     'all_grass/clm5'),
        ('all_grass',   '_JJA', 'all_grass/clm5'),
        ('all_grass',   '_MAM', 'all_grass/clm5'),
        ('all_forest_broadleaf',    '',     'all_forest_broadleaf/clm5'),
        ('all_forest_broadleaf',    '_JJA', 'all_forest_broadleaf/clm5'),
        ('all_forest_broadleaf',    '_MAM', 'all_forest_broadleaf/clm5'),
    ]

    data_vars = {}
    for scen, season, subdir in cases:
        key = f"{scen}{season}"
        data_vars[key] = _load_case_min(var, lat, lon, scen, season, subdir)
    
    return xr.Dataset(data_vars)

In [99]:
tair_N = get_min_data_aff_afb('TSA', lat, lon)
tsfc_N = get_min_data_aff_afb('TSKIN', lat, lon)
tatm_N = get_min_data_aff_afb('TBOT', lat, lon)

In [ ]:
# 1. Levels, tick‑labels and cmaps
levels_dict = {
    'seq': np.array([-5, 0, 5, 10, 15, 20, 25, 30]) + 273.15,
    'div': np.array([-2, -1, -0.5, -0.3, -0.1, 0.1, 0.3, 0.5, 1, 2])
}
ticks_dict = {
    "seq": ["\N{MINUS SIGN}5", "0", "5", "10", "15", "20", "25", "30"],
    "div": ['\N{MINUS SIGN}2.0', '\N{MINUS SIGN}1.0', '\N{MINUS SIGN}0.5', '\N{MINUS SIGN}0.3', '\N{MINUS SIGN}0.1', '+0.1', '+0.3', '+0.5', '+1.0', '+2.0']
}
cmaps = {'seq': 'hot_r', 'div': 'RdBu_r'}

unit = '$\\mathregular{^\\circ C}$'
vmin, vmax, extend = 0, 42, 'both'

# 2. Define each panel: (row, col), (var1, var0, levels‑type), title, letter
panels = [
    # row 0
    (0, 0, ('all_forest','control'),      'div', 'Aff–Ctl: $\\Delta$ $\mathregular{T_{sfc,yearN}}$', 'a'),
    (0, 1, ('all_forest_broadleaf','control'),'div','AfB–Ctl: $\\Delta$ $\mathregular{T_{sfc,yearN}}$','b'),
    (0, 2, ('all_forest_broadleaf','all_forest'),'div','AfB–Aff: $\\Delta$ $\mathregular{T_{sfc,yearN}}$','c'),
    (0, 3, ('all_forest_broadleaf','all_grass'),'div','AfB–Def: $\\Delta$ $\mathregular{T_{sfc,yearN}}$','d'),
    
    # row 1
    (1, 0, ('all_forest','control'),      'div', 'Aff–Ctl: $\\Delta$ $\mathregular{T_{air,yearN}}$','e'),
    (1, 1, ('all_forest_broadleaf','control'),'div','AfB–Ctl: $\\Delta$ $\mathregular{T_{air,yearN}}$','f'),
    (1, 2, ('all_forest_broadleaf','all_forest'),'div','AfB–Aff: $\\Delta$ $\mathregular{T_{air,yearN}}$','g'),
    (1, 3, ('all_forest_broadleaf','all_grass'),'div','AfB–Def: $\\Delta$ $\mathregular{T_{air,yearN}}$','h'),
    
    # row 2
    (2, 0, ('all_forest','control'),      'div', 'Aff–Ctl: $\\Delta$ $\mathregular{T_{atm,yearN}}$','i'),
    (2, 1, ('all_forest_broadleaf','control'),'div','AfB–Ctl: $\\Delta$ $\mathregular{T_{atm,yearN}}$','j'),
    (2, 2, ('all_forest_broadleaf','all_forest'),'div','AfB–Aff: $\\Delta$ $\mathregular{T_{atm,yearN}}$','k'),
    (2, 3, ('all_forest_broadleaf','all_grass'),'div','AfB–Def: $\\Delta$ $\mathregular{T_{atm,yearN}}$','l'),

]

# 3. Make the figure & axes grid
fig, axes = plt.subplots(
    3, 4,
    figsize=(12, 8),
    subplot_kw=dict(projection=map_proj, facecolor="lightgrey"),
    dpi=300
)
format_axes(axes.flatten())
fig.subplots_adjust(hspace=0.2, wspace=0.1, left=0.03, right=0.935, bottom=0.03, top=0.9)

# 4. Loop and plot
for row, col, (high, low), lvl_type, title, letter in panels:
    ax = axes[row, col]
    levels = levels_dict[lvl_type]
    cmap   = cmaps[lvl_type]
    ticks  = ticks_dict[lvl_type]

    diff = tsfc_N[high] - tsfc_N[low]  \
           if 'sfc' in title else \
           (tair_N if 'air' in title else tatm_N)[high] - (tair_N if 'air' in title else tatm_N)[low]

    opt = dict(
        transform=data_proj,
        vmin=vmin, vmax=vmax,
        levels=levels,
        extend=extend,
        add_colorbar=False,
        cmap=cmap
    )

    h = diff.plot(ax=ax, **opt)
    ax.set_title(f"{title} ({unit})")
    ax.add_feature(cfeature.OCEAN, color='whitesmoke')

    # only draw colorbar on rightmost column of each row
    if col <= 3:
        cbar = mpu.colorbar(
            h, ax,
            orientation='vertical',
            size=0.04,
            extend=extend,
            pad=0.05,
            ticks=levels
        )
        cbar.set_ticklabels(ticks)

    ax.text(
        0.02, 0.93, letter,
        color='k', fontsize=10,
        transform=ax.transAxes, weight='bold'
    )

In [ ]:
# 1. Levels, tick‑labels and cmaps
levels_dict = {
    'seq': np.array([-5, 0, 5, 10, 15, 20, 25, 30]) + 273.15,
    'div': np.array([-2, -1, -0.5, -0.3, -0.1, 0.1, 0.3, 0.5, 1, 2])
}
ticks_dict = {
    "seq": ["\N{MINUS SIGN}5", "0", "5", "10", "15", "20", "25", "30"],
    "div": ['\N{MINUS SIGN}2.0', '\N{MINUS SIGN}1.0', '\N{MINUS SIGN}0.5', '\N{MINUS SIGN}0.3', '\N{MINUS SIGN}0.1', '+0.1', '+0.3', '+0.5', '+1.0', '+2.0']
}
cmaps = {'seq': 'hot_r', 'div': 'RdBu_r'}

unit = '$\\mathregular{^\\circ C}$'
vmin, vmax, extend = 0, 42, 'both'

# 2. Define each panel: (row, col), (var1, var0, levels‑type), title, letter
panels = [
    # row 0
    (0, 0, ('all_forest_JJA','control_JJA'),      'div', 'Aff–Ctl: $\\Delta$ $\mathregular{T_{sfc,jjaN}}$', 'a'),
    (0, 1, ('all_forest_broadleaf_JJA','control_JJA'),'div','AfB–Ctl: $\\Delta$ $\mathregular{T_{sfc,jjaN}}$','b'),
    (0, 2, ('all_forest_broadleaf_JJA','all_forest_JJA'),'div','AfB–Aff: $\\Delta$ $\mathregular{T_{sfc,jjaN}}$','c'),
    (0, 3, ('all_forest_broadleaf_JJA','all_grass_JJA'),'div','AfB–Def: $\\Delta$ $\mathregular{T_{sfc,jjaN}}$','d'),
    
    # row 1
    (1, 0, ('all_forest_JJA','control_JJA'),      'div', 'Aff–Ctl: $\\Delta$ $\mathregular{T_{air,jjaN}}$','e'),
    (1, 1, ('all_forest_broadleaf_JJA','control_JJA'),'div','AfB–Ctl: $\\Delta$ $\mathregular{T_{air,jjaN}}$','f'),
    (1, 2, ('all_forest_broadleaf_JJA','all_forest_JJA'),'div','AfB–Aff: $\\Delta$ $\mathregular{T_{air,jjaN}}$','g'),
    (1, 3, ('all_forest_broadleaf_JJA','all_grass_JJA'),'div','AfB–Def: $\\Delta$ $\mathregular{T_{air,jjaN}}$','h'),
    
    # row 2
    (2, 0, ('all_forest_JJA','control_JJA'),      'div', 'Aff–Ctl: $\\Delta$ $\mathregular{T_{atm,jjaN}}$','i'),
    (2, 1, ('all_forest_broadleaf_JJA','control_JJA'),'div','AfB–Ctl: $\\Delta$ $\mathregular{T_{atm,jjaN}}$','j'),
    (2, 2, ('all_forest_broadleaf_JJA','all_forest_JJA'),'div','AfB–Aff: $\\Delta$ $\mathregular{T_{atm,jjaN}}$','k'),
    (2, 3, ('all_forest_broadleaf_JJA','all_grass_JJA'),'div','AfB–Def: $\\Delta$ $\mathregular{T_{atm,jjaN}}$','l'),

]

# 3. Make the figure & axes grid
fig, axes = plt.subplots(
    3, 4,
    figsize=(12, 8),
    subplot_kw=dict(projection=map_proj, facecolor="lightgrey"),
    dpi=300
)
format_axes(axes.flatten())
fig.subplots_adjust(hspace=0.2, wspace=0.1, left=0.03, right=0.935, bottom=0.03, top=0.9)

# 4. Loop and plot
for row, col, (high, low), lvl_type, title, letter in panels:
    ax = axes[row, col]
    levels = levels_dict[lvl_type]
    cmap   = cmaps[lvl_type]
    ticks  = ticks_dict[lvl_type]

    diff = tsfc_N[high] - tsfc_N[low]  \
           if 'sfc' in title else \
           (tair_N if 'air' in title else tatm_N)[high] - (tair_N if 'air' in title else tatm_N)[low]

    opt = dict(
        transform=data_proj,
        vmin=vmin, vmax=vmax,
        levels=levels,
        extend=extend,
        add_colorbar=False,
        cmap=cmap
    )

    h = diff.plot(ax=ax, **opt)
    ax.set_title(f"{title} ({unit})")
    ax.add_feature(cfeature.OCEAN, color='whitesmoke')

    # only draw colorbar on rightmost column of each row
    if col <= 3:
        cbar = mpu.colorbar(
            h, ax,
            orientation='vertical',
            size=0.04,
            extend=extend,
            pad=0.05,
            ticks=levels
        )
        cbar.set_ticklabels(ticks)

    ax.text(
        0.02, 0.93, letter,
        color='k', fontsize=10,
        transform=ax.transAxes, weight='bold'
    )

In [ ]:
# 1. Levels, tick‑labels and cmaps
levels_dict = {
    'seq': np.array([-5, 0, 5, 10, 15, 20, 25, 30]) + 273.15,
    'div': np.array([-2, -1, -0.5, -0.3, -0.1, 0.1, 0.3, 0.5, 1, 2])
}
ticks_dict = {
    "seq": ["\N{MINUS SIGN}5", "0", "5", "10", "15", "20", "25", "30"],
    "div": ['\N{MINUS SIGN}2.0', '\N{MINUS SIGN}1.0', '\N{MINUS SIGN}0.5', '\N{MINUS SIGN}0.3', '\N{MINUS SIGN}0.1', '+0.1', '+0.3', '+0.5', '+1.0', '+2.0']
}
cmaps = {'seq': 'hot_r', 'div': 'RdBu_r'}

unit = '$\\mathregular{^\\circ C}$'
vmin, vmax, extend = 0, 42, 'both'

# 2. Define each panel: (row, col), (var1, var0, levels‑type), title, letter
panels = [
    # row 0
    (0, 0, ('all_forest_MAM','control_MAM'),      'div', 'Aff–Ctl: $\\Delta$ $\mathregular{T_{sfc,mamN}}$', 'a'),
    (0, 1, ('all_forest_broadleaf_MAM','control_MAM'),'div','AfB–Ctl: $\\Delta$ $\mathregular{T_{sfc,mamN}}$','b'),
    (0, 2, ('all_forest_broadleaf_MAM','all_forest_MAM'),'div','AfB–Aff: $\\Delta$ $\mathregular{T_{sfc,mamN}}$','c'),
    (0, 3, ('all_forest_broadleaf_MAM','all_grass_MAM'),'div','AfB–Def: $\\Delta$ $\mathregular{T_{sfc,mamN}}$','d'),
    
    # row 1
    (1, 0, ('all_forest_MAM','control_MAM'),      'div', 'Aff–Ctl: $\\Delta$ $\mathregular{T_{air,mamN}}$','e'),
    (1, 1, ('all_forest_broadleaf_MAM','control_MAM'),'div','AfB–Ctl: $\\Delta$ $\mathregular{T_{air,mamN}}$','f'),
    (1, 2, ('all_forest_broadleaf_MAM','all_forest_MAM'),'div','AfB–Aff: $\\Delta$ $\mathregular{T_{air,mamN}}$','g'),
    (1, 3, ('all_forest_broadleaf_MAM','all_grass_MAM'),'div','AfB–Def: $\\Delta$ $\mathregular{T_{air,mamN}}$','h'),
    
    # row 2
    (2, 0, ('all_forest_MAM','control_MAM'),      'div', 'Aff–Ctl: $\\Delta$ $\mathregular{T_{atm,mamN}}$','i'),
    (2, 1, ('all_forest_broadleaf_MAM','control_MAM'),'div','AfB–Ctl: $\\Delta$ $\mathregular{T_{atm,mamN}}$','j'),
    (2, 2, ('all_forest_broadleaf_MAM','all_forest_MAM'),'div','AfB–Aff: $\\Delta$ $\mathregular{T_{atm,mamN}}$','k'),
    (2, 3, ('all_forest_broadleaf_MAM','all_grass_MAM'),'div','AfB–Def: $\\Delta$ $\mathregular{T_{atm,mamN}}$','l'),

]

# 3. Make the figure & axes grid
fig, axes = plt.subplots(
    3, 4,
    figsize=(12, 8),
    subplot_kw=dict(projection=map_proj, facecolor="lightgrey"),
    dpi=300
)
format_axes(axes.flatten())
fig.subplots_adjust(hspace=0.2, wspace=0.1, left=0.03, right=0.935, bottom=0.03, top=0.9)

# 4. Loop and plot
for row, col, (high, low), lvl_type, title, letter in panels:
    ax = axes[row, col]
    levels = levels_dict[lvl_type]
    cmap   = cmaps[lvl_type]
    ticks  = ticks_dict[lvl_type]

    diff = tsfc_N[high] - tsfc_N[low]  \
           if 'sfc' in title else \
           (tair_N if 'air' in title else tatm_N)[high] - (tair_N if 'air' in title else tatm_N)[low]

    opt = dict(
        transform=data_proj,
        vmin=vmin, vmax=vmax,
        levels=levels,
        extend=extend,
        add_colorbar=False,
        cmap=cmap
    )

    h = diff.plot(ax=ax, **opt)
    ax.set_title(f"{title} ({unit})")
    ax.add_feature(cfeature.OCEAN, color='whitesmoke')

    # only draw colorbar on rightmost column of each row
    if col <= 3:
        cbar = mpu.colorbar(
            h, ax,
            orientation='vertical',
            size=0.04,
            extend=extend,
            pad=0.05,
            ticks=levels
        )
        cbar.set_ticklabels(ticks)

    ax.text(
        0.02, 0.93, letter,
        color='k', fontsize=10,
        transform=ax.transAxes, weight='bold'
    )